In [3]:
import numpy as np
import pandas as pd

In [4]:
data = {
    "Model": ["GPT-3.5", "T5-Large", "BERT Fine-Tuned"],
    "Perplexity": [15, 18, 20],  # Lower is better
    "BLEU": [0.92, 0.89, 0.87],  # Higher is better
    "ROUGE": [0.85, 0.81, 0.79],  # Higher is better
    "Execution Time": [3.2, 4.5, 5.1],  # Lower is better
    "Memory Usage": [1024, 768, 896]  # Lower is better
}


df = pd.DataFrame(data)

In [5]:
criteria = ["Perplexity", "BLEU", "ROUGE", "Execution Time", "Memory Usage"]
benefit_criteria = ["BLEU", "ROUGE"]  # Higher is better
cost_criteria = ["Perplexity", "Execution Time", "Memory Usage"]  # Lower is better

In [6]:
normalized_data = df.copy()
for criterion in criteria:
    if criterion in benefit_criteria:
        normalized_data[criterion] = df[criterion] / np.sqrt(np.sum(df[criterion]**2))
    elif criterion in cost_criteria:
        normalized_data[criterion] = (1 / df[criterion]) / np.sqrt(np.sum((1 / df[criterion])**2))

In [7]:
weights = {
    "Perplexity": 0.25,
    "BLEU": 0.25,
    "ROUGE": 0.25,
    "Execution Time": 0.15,
    "Memory Usage": 0.10
}

def weighted_normalized(df, weights):
    for col in weights:
        df[col] = df[col] * weights[col]
    return df

weighted_data = weighted_normalized(normalized_data.copy(), weights)

In [8]:
ideal_solution = {}
negative_ideal_solution = {}
for criterion in criteria:
    ideal_solution[criterion] = weighted_data[criterion].max() if criterion in benefit_criteria else weighted_data[criterion].min()
    negative_ideal_solution[criterion] = weighted_data[criterion].min() if criterion in benefit_criteria else weighted_data[criterion].max()

In [9]:
def calculate_distance(df, solution):
    distance = []
    for _, row in df.iterrows():
        dist = np.sqrt(np.sum([(row[criterion] - solution[criterion])**2 for criterion in criteria]))
        distance.append(dist)
    return np.array(distance)

weighted_data["Positive Distance"] = calculate_distance(weighted_data, ideal_solution)
weighted_data["Negative Distance"] = calculate_distance(weighted_data, negative_ideal_solution)

In [10]:
weighted_data["TOPSIS Score"] = weighted_data["Negative Distance"] / (
    weighted_data["Positive Distance"] + weighted_data["Negative Distance"]
)

In [11]:
weighted_data["Rank"] = weighted_data["TOPSIS Score"].rank(ascending=False)
results = weighted_data[["Model", "TOPSIS Score", "Rank"]]
results.to_csv("topsis_results.csv", index=False)